In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 

import cptac
import cptac.utils as u

In [2]:
c1 = cptac.Endometrial()
c2 = cptac.Ovarian()

In [3]:
cancer1 = "Endometrial"
cancer2 = "Ovarian"
gene = 'TP53'

In [4]:
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
prot = c1.get_proteomics()
prot_list = list(prot.columns)
prot_and_mutations = c1.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = prot_list)  

# Step 2 - Create binary column 
prot_and_mutations['binary_mutations'] = np.where(
            prot_and_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = prot_and_mutations[prot_and_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status = mut_status.dropna(axis=1,how='all')

In [5]:
col_list = list(mut_status.columns)
col_list.remove('binary_mutations')

print("Doing t-test comparisons for", cancer1)

# Call wrap_ttest, pass in formatted dataframe
wrap_results = u.wrap_ttest(mut_status, 'binary_mutations', col_list, return_all=True)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Doing t-test comparisons for Endometrial
135 significant comparisons!


In [6]:
print(wrap_results.loc[wrap_results['Comparison'] == gene+"_proteomics"])
wrap_results

         Comparison       P_Value
64  TP53_proteomics  3.991072e-07


,Comparison,P_Value
0,MTHFD2_proteomics,9.298849e-12
1,SLC4A1AP_proteomics,5.983585e-11
2,XPO5_proteomics,1.551177e-10
3,TRMT6_proteomics,1.801163e-10
4,STAT2_proteomics,1.922167e-10
...,...,...
10988,KCTD13_proteomics,9.885735e-01
10989,TSSC4_proteomics,9.923845e-01
10990,KRCC1_proteomics,9.942184e-01
10991,METTL7B_proteomics,9.995390e-01


In [20]:
# reduce multiindex for ovarian proteomics
# Step 1 - Create dataframe in order to do comparisons with wrap_ttest
prot2 = c2.get_proteomics()
prot2 = c2.reduce_multiindex(prot2, levels_to_drop=['Database_ID'], flatten=True)
#prot2.columns.value_counts()

In [26]:
prot_list2 = list(prot2.columns)
prot_and_mutations = c2.join_omics_to_mutations(
    mutations_genes = [gene], omics_df_name = 'proteomics', omics_genes = prot_list2) 
prot_and_mutations = c2.reduce_multiindex(prot_and_mutations, flatten=True)
prot_and_mutations

Name,A1BG_proteomics_NP_570602,A2M_proteomics_NP_000005,A2ML1_proteomics_NP_653271,AAAS_proteomics_NP_056480,AACS_proteomics_NP_076417,AAGAB_proteomics_NP_078942,AAK1_proteomics_NP_055726,AAMDC_proteomics_NP_078960,AAMP_proteomics_NP_001078,AAR2_proteomics_NP_001258803,...,ZWILCH_proteomics_NP_060445,ZWINT_proteomics_NP_008988,ZYG11B_proteomics_NP_078922,ZYX_proteomics_NP_001010972,ZZEF1_proteomics_NP_055928,ZZZ3_proteomics_NP_056349,TP53_Mutation,TP53_Location,TP53_Mutation_Status,Sample_Status
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S002,0.133634,0.765120,-0.778849,0.118236,0.112740,0.054942,0.313055,-0.786270,-0.384680,0.064017,...,-0.195793,NaN,-0.145053,0.590217,0.019414,NaN,[Frame_Shift_Del],[p.D61*],Single_mutation,Tumor
S006,-0.432786,-0.461906,-0.533497,0.041800,0.604114,0.351294,0.546605,0.603935,-0.235374,0.059084,...,0.574339,NaN,0.075401,-0.437564,-0.208831,NaN,[Frame_Shift_Del],[p.S90Vfs*55],Single_mutation,Tumor
S007,-0.013943,0.065842,NaN,-0.010514,0.541935,0.169826,-0.023032,-0.057549,-0.397627,0.199581,...,1.020321,NaN,-0.245864,0.008411,0.130504,-0.367433,[Missense_Mutation],[p.R273C],Single_mutation,Tumor
S009,1.234203,1.512082,-0.703050,0.181152,0.087719,0.226861,-0.100766,-0.516022,-0.456512,-0.257629,...,0.149523,-0.313370,-0.507473,0.221818,0.278313,-1.008998,[Missense_Mutation],[p.I195T],Single_mutation,Tumor
S011,-0.482872,0.343872,-1.150936,0.315155,0.097235,-0.160208,-0.125165,-0.768776,-0.372368,0.450086,...,0.007854,-0.106049,0.377063,-0.295592,0.011997,-0.236003,[Frame_Shift_Ins],[p.C124Lfs*25],Single_mutation,Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S127,-0.012139,0.437837,-0.704253,0.411204,-0.418927,0.016434,0.504605,0.022115,-0.163317,0.290281,...,-0.612827,-0.149425,0.017523,0.442398,0.088551,-0.614771,[Wildtype_Normal],[No_mutation],Wildtype_Normal,Normal
S128,1.949941,1.803647,0.924203,-0.025210,0.593528,-0.427610,-0.094656,0.082444,0.065310,-0.322335,...,0.716779,-0.676827,-0.569823,0.640503,0.763645,-1.370872,[Wildtype_Normal],[No_mutation],Wildtype_Normal,Normal
S129,1.523833,3.418092,0.281521,-0.148852,-0.051749,0.297993,0.160205,-0.318083,-0.382827,0.420826,...,-0.048390,NaN,0.675466,0.267934,0.583772,0.501873,[Wildtype_Normal],[No_mutation],Wildtype_Normal,Normal


In [27]:
# Step 2 - Create binary column 
prot_and_mutations['binary_mutations'] = np.where(
            prot_and_mutations[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

# Step 3 - Format the dataframe correctly for the T-test(just omics and binary columns for tumors)
tumors = prot_and_mutations[prot_and_mutations.Sample_Status == 'Tumor'] #drop Normal samples
columns_to_drop = [gene+"_Mutation", gene+"_Location", gene+"_Mutation_Status", "Sample_Status"]
mut_status = tumors.drop(columns_to_drop, axis = 1)
mut_status2 = mut_status.dropna(axis=1,how='all')

In [28]:
col_list2 = list(mut_status2.columns)
col_list2.remove('binary_mutations')

print("Doing t-test comparisons for", cancer2)

# Call wrap_ttest, pass in formatted dataframe
wrap_results2 = u.wrap_ttest(mut_status2, 'binary_mutations', col_list2, return_all=True)

Doing t-test comparisons for Ovarian
No significant comparisons.


In [29]:
print(wrap_results2.loc[wrap_results2['Comparison'] == gene+"_proteomics"])
wrap_results2

Empty DataFrame
Columns: [Comparison, P_Value]
Index: []


,Comparison,P_Value
0,C15orf41_proteomics_NP_001123482,0.001754
1,C20orf24_proteomics_NP_001186463,0.009271
2,CALCRL_proteomics_NP_005786,0.010477
3,CAB39L_proteomics_NP_001274267,0.013414
4,BTBD2_proteomics_NP_060267,0.014760
...,...,...
10681,URI1_proteomics_NP_001239570,0.996227
10682,RPGR_proteomics_NP_001030025,0.996601
10683,S100A7_proteomics_NP_002954,0.998084
10684,ZNF768_proteomics_NP_078947,0.998641


In [30]:
#combine pval results
combined = wrap_results.merge(wrap_results2, on='Comparison')
combined = combined.replace(regex=True,to_replace='_proteomics', value='')

In [31]:
# Green = interacting protein
interacting_list = u.get_interacting_proteins(gene)
combined["color"] = np.where(combined["Comparison"].isin(interacting_list), "green", "black")
print(interacting_list)
#combined.loc[combined['color'] == 'green']

['RPA1', 'AURKA', 'MDM2', 'SIRT1', 'CREBBP', 'EP300', 'CDK2', 'TP53', 'ATM', 'BCL2L1', 'TP53BP2', 'CCNG1', 'MDM4', 'PTEN', 'TP53BP1', 'CHEK2', 'FZR1', 'MAPK8', 'KDM1A', 'BCL6', 'CDKN1A', 'HIPK2', 'CHEK1', 'CDKN2A', 'BRCA1', 'AKT1', 'DVL2', 'BLM', 'ATR', 'TFAP2A', 'EPHA3']


In [32]:
limit_x = combined.loc[combined['P_Value_x'] <= .05]
limit_y = combined.loc[combined['P_Value_y'] <= .05]
limit_both = limit_x.loc[limit_x['P_Value_y'] <= .05]
print(limit_both.loc[limit_both['color'] == 'green'])
limit_both

Empty DataFrame
Columns: [Comparison, P_Value_x, P_Value_y, color]
Index: []


,Comparison,P_Value_x,P_Value_y,color


In [ ]:
# Format figure size and font
plt.rcParams['figure.figsize']=(15,10)
sns.set(font_scale = 1.3)

# Format title and axis
plot = sns.regplot(x="P_Value_x", y="P_Value_y", data=limit_both, scatter_kws={'facecolors':limit_both['color']})
plot.set_title("P_Value Distribution for "+gene+" trans Proteomics")
plot.set(xlabel = 'P-Value for '+cancer1)
plot.set(ylabel = 'P-Value for '+cancer2)

plt.show()
plt.clf()
plt.close()

ARID1A: 8 significant results for endometrial. No significant results for colon. 4 interacting proteins in graph with alpha = 0.05 borders.